### import libraries

In [2]:
import geopandas as gpd
from shapely.geometry import box, Polygon, MultiPolygon


### Read files
- Read gpkg Lucas survey to get the poligons
- Dataset https://data.jrc.ec.europa.eu/dataset/e3fe3cd0-44db-470e-8769-172a8b9e8874#dataaccess

In [6]:
gdf = gpd.read_file("dataset/l2022_survey_cop_radpoly_attr.gpkg")

In [ ]:
# Convert to kml first ro

gdf[gdf["point_id"] == 47081563].to_file("test.kml", driver="KML")

### Filter agriculutral data
- Filter the U111 agricultural data found n dataset documentation

In [ ]:
agri = gdf[gdf["lu1_code"] == "U111"]

In [ ]:
# Override the existing CRS (force setting without transformation)
#agri.set_crs(epsg=32633, allow_override=True, inplace=True)


# Convert to a different CRS, e.g., WGS84 (EPSG:4326) - Latitude/Longitude
agri = agri.to_crs(epsg=4326)

### Get la Italy region

In [ ]:
world = gpd.read_file(
    "politcal_reference/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp"
)
italy = world[world["NAME"] == "Italy"]

if agri.crs != italy.crs:
    agri = agri.to_crs(italy.crs)


In [ ]:
agri_italy = gpd.sjoin(agri, italy, how="inner", predicate="intersects")

### Get Tuscany region
- Still working on it.

In [ ]:
italy = gpd.read_file(
    "politcal_reference/italy_political_regions/Com01012019_WGS84.shp"
)
tuscanny = italy[italy["COD_REG"] == 9]

In [ ]:
tuscanny = tuscanny.to_crs(epsg=4326)

In [ ]:
# Find all the poligons that intersect with any tuscanny municipality
if 'index_right' in agri_italy.columns:
    agri_italy = agri_italy.rename(columns={'index_right': 'agri_index_right'})

if 'index_right' in tuscanny.columns:
    tuscanny = tuscanny.rename(columns={'index_right': 'tuscanny_index_right'})

agri_tuscanny = gpd.sjoin(agri_italy, tuscanny, how="inner", predicate="intersects")

### Export poligons

In [ ]:
agri.to_file("dataset/l2022_survey_agri.gpkg", driver="GPKG")
agri_italy.to_file("dataset/l2022_survey_agri_italy.gpkg", driver="GPKG")
agri_tuscanny.to_file("dataset/l2022_survey_agri_toscana.gpkg", driver="GPKG")

### Put the poligons on map.

In [ ]:
import geopandas as gpd
import folium

# Assuming 'agri' is your GeoDataFrame with polygons
# Convert the CRS to EPSG:4326 for folium compatibility
italy_agri = agri_italy.to_crs(epsg=4326)

count = 0
# Create a base map centered around the centroid of the polygons
m = folium.Map(
    location=[
        italy_agri.geometry.centroid.y.mean(),
        agri.geometry.centroid.x.mean(),
    ],
    zoom_start=12,
)

# Add polygons to the map
for _, row in italy_agri.iterrows():
    print(count)
    count += 1
    # Convert each geometry to a GeoJSON format for Folium
    geo_json = folium.GeoJson(row['geometry'],
                              style_function=lambda x: {
                                  'fillColor': row['col_hex'],  # The fill color of the polygon
                                  'color': 'black',  # The border color of the polygon
                                  'weight': 3,  # The thickness of the border
                                  'fillOpacity': 0.8  # Opacity of the fill color
                              })
    geo_json.add_to(m)

# Display the map
m.save('agri_polygons_map.html')